# Create a configuration cell to load a data cube

*****

The geographic extent, time period, product and measurements that you want to use will differ depending on your goals. This notebook helps you to create a configuration cell to be copy/pasted or loaded into a notebook for your analysis.

In [1]:
# Make sure the script is using the proper kernel
try:
    %run ../swiss_utils/assert_env.py
except:
    %run ./swiss_utils/assert_env.py

In [2]:
# Import modules

# reload module before executing code
%load_ext autoreload
%autoreload 2

# define modules locations (you might have to adapt define_mod_locs.py)
%run ../swiss_utils/define_mod_locs.py

import ipywidgets as widgets

from shapely.geometry import Polygon

from swiss_utils.data_cube_utilities.sdc_utilities import new_get_query_metadata
from swiss_utils.data_cube_utilities.sdc_advutils import draw_map

import datacube
dc = datacube.Datacube()

import utils.data_cube_utilities.data_access_api as dc_api  
api = dc_api.DataAccessApi()

In [3]:
# Allow user to select an available product prioritizing ingested (if any)over indexed

product_names = []

products = dc.list_products()
prfxs = set(prod.split('_')[0] for prod in products['name'])
for prfx in prfxs:
    for prprods in [[x for x in products['name'] if x.startswith(prfx)]]:
        # print(prprods)
        if len(prprods) == 1:
            product_names.append(prprods[0])
        else:
            swissprod = [x for x in prprods if x.endswith('_swiss')]
            if len(swissprod) == 1:
                product_names.append(swissprod[0])
            else:
                product_names.extend(prprods)
product_names.sort()
                
# Select the product
product_sel = widgets.RadioButtons(options=product_names, disabled=False)
display(widgets.Label('Select a product and run the next cell: '), product_sel)

Label(value='Select a product and run the next cell: ')

RadioButtons(options=('arealstatistik', 'astgtmv003_scene', 'combiprecip_scene', 'corinelc_europe', 'corinelc_…

In [4]:
# Select measurements

measurement_list = dc.list_measurements(with_pandas=False)
measurements_for_product = filter(lambda x: x['product'] == product_sel.value, measurement_list)
valid_measurements_name_array = set(map(lambda x: x['name'], measurements_for_product))
# Landsat and Sentinel 2
try:
    measur_sel = widgets.SelectMultiple(options=sorted(list(valid_measurements_name_array)),
                                        value = ['blue', 'green', 'red'],
                                        disabled=False)
# other products
except:
    measur_sel = widgets.SelectMultiple(options=sorted(list(valid_measurements_name_array)),
                                        disabled=False)
display(widgets.Label('Select a measurements and run the next cell: '), measur_sel)

Label(value='Select a measurements and run the next cell: ')

SelectMultiple(index=(1, 3, 7), options=('aerosol_qa', 'blue', 'coastal_aerosol', 'green', 'nir', 'pixel_qa', …

In [5]:
# Convert tupple to list and add mask band if required

measur_sel = list(measur_sel.value)

if 'pixel_qa' not in measur_sel and 'pixel_qa' in list(valid_measurements_name_array):
    print('pixel_qa measurement added')
    measur_sel.append('pixel_qa')
elif 'slc' not in measur_sel and 'slc' in list(valid_measurements_name_array):
    print('slc measurement added')
    measur_sel.append('slc')

pixel_qa measurement added


In [6]:
# Select geographical extent

mtd = new_get_query_metadata(dc, product_sel.value)
# if too slow use the line below (will not be accurate with all product
# mtd = new_get_query_metadata(dc, product_sel.value, quick = True)

full_lat = mtd['lat_extents']
full_lon = mtd['lon_extents']
min_max_dates = mtd['time_extents']
crs = mtd['crs']

# Add an empty map you can draw on it
m, poly = draw_map(full_lat, full_lon, crs)
print('Zoom, pan and draw a rectangle or polygon (the bounding box will be used) and run the next cell:')
m

Zoom, pan and draw a rectangle or polygon (the bounding box will be used) and run the next cell:


Map(center=[46.73757106238716, 8.506320734024964], controls=(ZoomControl(options=['position', 'zoom_in_text', …

In [11]:
# Once a feature is drawn, extract the bounding box of the last feature drawn

coords = poly.last_draw['geometry']['coordinates']
geo_extent = Polygon(coords[0]).bounds

min_lon = geo_extent[0]
min_lat = geo_extent[1]
max_lon = geo_extent[2]
max_lat = geo_extent[3]

In [14]:
# Select time period

start_date = widgets.DatePicker(description='Start date',
                                value = min_max_dates[0].date(),
                                disabled=False)
end_date = widgets.DatePicker(description='End date',
                              value = min_max_dates[1].date(),
                              disabled=False)
display(widgets.Label('IF REQUIRED define time period (cannot be outside of the initial displayed time) and run the next cell:'),
        widgets.HBox([start_date, end_date]))

Label(value='IF REQUIRED define time period (cannot be outside of the initial displayed time) and run the next…

In [15]:
    # Check defined time period

from datetime import timedelta
end_date = end_date.value + timedelta(days=1) # end_date is not inclusive !

assert start_date.value >= min_max_dates[0].date(), \
       'Start date cannot be defined before {}'.format(min_max_dates[0].date())
assert end_date - timedelta(days=1) <= min_max_dates[1].date(), \
       'End date cannot be defined after {}'.format(min_max_dates[1].date())
assert start_date.value < end_date, \
       'End date is defined before start date'

print('Time period is OK')

Time period is OK


In [16]:
# Resume configuration parameters in a format ready to be copy/pasted to a new cell,
# and in a txt file to be loaded with the '%load config_cell.txt' magic.

str = '''# Configuration

product = '{}'
measurements = {}

min_lon = {}
max_lon = {}
min_lat = {}
max_lat = {}

start_date = datetime.strptime('{}', '%Y-%m-%d')
end_date = datetime.strptime('{}', '%Y-%m-%d')'''.\
format(product_sel.value,
       measur_sel,
       min_lon, max_lon, min_lat, max_lat,
       start_date.value.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d'))
print(str)
with open('config_cell.txt', 'w') as text_file:
    print(str, file=text_file)

# Configuration

product = 'ls8_lasrc_swiss'
measurements = ['blue', 'green', 'red', 'pixel_qa']

min_lon = 7.109528
max_lon = 7.189178
min_lat = 46.795359
max_lat = 46.851974

start_date = datetime.strptime('2013-03-22', '%Y-%m-%d')
end_date = datetime.strptime('2021-12-30', '%Y-%m-%d')
